In [110]:
%%capture
%pip install pandas_read_xml
%pip install --upgrade pip
!pip3 install upgrade-pip

In [111]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import numpy as np
#import sys
#print(sys.version)

In [112]:
def getStats(df,columnIndex):
    
    meanVar=round(df.iloc[:,columnIndex].mean(),3)
    statList=[meanVar,
              round(df.iloc[:,columnIndex].max(),3),
              round(df.iloc[:,columnIndex].min(),3),
              round(df.iloc[:,columnIndex].median(),3),
              round(df.iloc[:,columnIndex].quantile(0.25),3),
              round(df.iloc[:,columnIndex].quantile(0.75),3),
              round(df.iloc[:,columnIndex].std()/abs(meanVar),3)
             ]                   
    
    return statList

In [113]:
def getItemCounts(df,columnIndex,List):

    result=[]

    for item in List:
        result.append((df.iloc[:,columnIndex]==item).sum())

    return result

In [114]:
def getRates1(List,divisor):
    
    Rates=[]

    for item in List:
        Rates.append(round(item/divisor,3))
    
    return Rates

In [115]:
def getRates2(List,Dict,divisor):
    
    Rates=[]

    for item in List:
        Rates.append(round(sum([Dict[i] for i in item])/divisor,3))       
    
    return Rates

In [116]:
def getCharRates(df,colIndex,charList,sumLenWord):
    
    charRates=[]
    
    for char in charList:    
        charRates.append(round(df.iloc[:,colIndex].str.count(char).sum()/sumLenWord,3))
    
    return charRates

In [117]:
def getAuthorName(poems_path):
    
    try:
    
        df0 = pd.read_xml(poems_path,xpath="//ns:titleStmt/ns:author/ns:persName",namespaces={"ns":"http://www.tei-c.org/ns/1.0"})

        Author=df0.iloc[0, 0]+' '+df0.iloc[0, 1]

        AuthorDict={'Author':Author}

        return AuthorDict
    
    except Exception as e:
        AuthorDict={'Author':np.nan}
        return AuthorDict

In [118]:
def getDiv(poems_path):
    
    try:

        divDict={}

        divKeyList1=['Title','numWordsTitle']
        divKeyList2=['numStanza','numLine','numWord','numSyll']
        divKeyList3=['rateShortSyll']

        df1 = pd.read_xml(poems_path, xpath="//div")

        special_characters=['!',':',';','"','-','.','?',',','(',')','[',']','»','«','`']

        divDict[divKeyList1[0]]=df1.iloc[0, 9]
        divDict[divKeyList1[1]]=len(divDict['Title'].replace(f"[{''.join(special_characters)}]",'').split())

        divList=[]

        for i in range(1,5):
            divList.append(df1.iloc[0,i])

        divDict.update(dict(zip(divKeyList2,divList)))
        divDict[divKeyList3[0]]=round(df1.iloc[0, 5] / divDict['numSyll'],3)

        return divDict

    except Exception as e:
        divDict=dict.fromkeys(divKeyList1+divKeyList2+divKeyList3,np.nan)
        return divDict

In [119]:
def getLg(poems_path):
    
    lgKeyList1=['meanNL','maxNL','minNL','medNL','q1NL','q3NL','vNL']
    lgKeyList2=['meanNW','maxNW','minNW','medNW','q1NW','q3NW','vNW']
    lgKeyList3=['meanNS','maxNS','minNS','medNS','q1NS','q3NS','vNS']
    lgKeyList4=['meanNshortS','maxNshortS','minNshortS','medNshortS','q1NshortS','q3NshortS','vNshortS']
    lgKeyList5=['meanNlongS','maxNlongS','minNlongS','medNlongS','q1NlongS','q3NlongS','vNlongS']
    lgKeyList6=['abcb','aba','abcdb','abca','abb','aaaa']
    
    keyList=[lgKeyList1,lgKeyList2,lgKeyList3,lgKeyList4,lgKeyList5]

    try:

        lgDict={}

        df2 = pd.read_xml(poems_path, xpath="//lg")
        
        for i in range(1,6):
            lgDict.update(dict(zip(keyList[i-1],getStats(df2,i))))

        rhymeFormCountList=getItemCounts(df2,6,lgKeyList6)
        lgDict.update(dict(zip(lgKeyList6,rhymeFormCountList)))

        return lgDict

    except Exception as e:
        lgDict=dict.fromkeys(lgKeyList1+lgKeyList2+lgKeyList3+lgKeyList4+lgKeyList5+lgKeyList6,np.nan)
        return lgDict

In [120]:
def getL(poems_path):
    
    lKeyList1=['LmeanNW','LmaxNW','LminNW','LmedNW','Lq1NW','Lq3NW','LvNW']
    lKeyList2=['LmeanNS','LmaxNS','LminNS','LmedNS','Lq1NS','Lq3NS','LvNS']
    lKeyList3=['LmeanNshortS','LmaxNshortS','LminNshortS','LmedNshortS','Lq1NshortS','Lq3NshortS','LvNshortS']
    lKeyList4=['LmeanNlongS','LmaxNlongS','LminNlongS','LmedNlongS','Lq1NlongS','Lq3NlongS','LvNlongS']
    
    keyList=[lKeyList1,lKeyList2,lKeyList3,lKeyList4]

    try:
    
        lDict={}

        df3 = pd.read_xml(poems_path, xpath="//l")
        
        for i in range(1,5):
            lDict.update(dict(zip(keyList[i-1],getStats(df3,i))))

        return lDict

    except Exception as e:
        lDict=dict.fromkeys(lKeyList1+lKeyList2+lKeyList3+lKeyList4,np.nan)
        return lDict

In [121]:
def getW(poems_path):
    
    wKeyList1=['PROPN','NOUN','ADJ','NUM','PRON','VERB','ADV','CONJ','SCONJ','DET','ADP','INTJ','PART']
    wKeyList2=['ratePROPN_NOUN','rateADJ','rateNOUNs','rateVERB','rateADV']
    wKeyList3=['ratePhonTlow','ratePhonThigh','ratePhonTmixed']
    wKeyList4=['meanLenWord','maxLenWord','minLenWord','medLenWord','q1LenWord','q3LenWord','vLenWord']
    wKeyList5=['sumLenWord']
    wKeyList6=['rateConsonant','rateShortFFV','rateLongFFV','rateShortBFV','rateLongBFV']    
    
    try:

        wDict={}

        df1 = pd.read_xml(poems_path, xpath="//div")
        numWord=df1.iloc[0, 3]

        df4 = pd.read_xml(poems_path, xpath="//w")
        
        nounsCountList=getItemCounts(df4,2,wKeyList1)
        wDict.update(dict(zip(wKeyList1,nounsCountList)))

        nounsList=[['PROPN','NOUN'],['ADJ'],['PROPN','NOUN','ADJ','NUM','PRON'],['VERB'],['ADV']]
        wDict.update(dict(zip(wKeyList2,getRates2(nounsList,wDict,numWord))))

        vowelsList=['low','high','mixed']
        vowelsCountList=getItemCounts(df4,5,vowelsList)
        wDict.update(dict(zip(wKeyList3,getRates1(vowelsCountList,numWord))))

        df4['lenWord']=df4.iloc[:, 6].str.len()
        wDict.update(dict(zip(wKeyList4,getStats(df4,8))))

        wDict[wKeyList5[0]]=df4.iloc[:,8].sum()
        charList=['c','f','F','b','B']
        wDict.update(dict(zip(wKeyList6,getCharRates(df4,6,charList,wDict['sumLenWord']))))

        return wDict
    
    except Exception as e:
        wDict=dict.fromkeys(wKeyList1+wKeyList2+wKeyList3+wKeyList4+wKeyList5+wKeyList6,np.nan)
        return wDict

In [122]:
def getPC(poems_path):
    
    try:
        
        df5 = pd.read_xml(poems_path, xpath="//pc")

        PUNCT=(df5.iloc[:, 1]=='PUNCT').sum()

        PUNCTdict={'PUNCT':PUNCT}

        return PUNCTdict
    
    except Exception as e:
        PUNCTdict={'PUNCT':np.nan}
        return PUNCTdict

In [123]:
def getRhymePair(poems_path):
    
    try:

        df6 = pd.read_xml(poems_path, xpath="//rhymePair")

        numRhymePairs=df6.iloc[:, 2].count()

        rhymePairDict={'numRhymePairs':numRhymePairs}

        return rhymePairDict
    
    except Exception as e:
        rhymePairDict={'numRhymePairs':np.nan}
        return rhymePairDict

In [124]:
def getAlliteration(poems_path):
    
    allKeyList1=['numAll','rateClearAll']
    allKeyList2=['meanAll','maxAll','minAll','medAll','q1All','q3All','vAll']
    
    try:
        
        AllDict={}

        df7 = pd.read_xml(poems_path, xpath="//alliteration")

        AllDict[allKeyList1[0]]=df7.iloc[:, 1].count()

        df7['lenAll']=df7.iloc[:, 1].apply(len)
        AllDict[allKeyList1[1]]=round((df7.iloc[:, 1].str.count('a')==df7.iloc[:, 6]).sum()/AllDict['numAll'],3)

        AllDict.update(dict(zip(allKeyList2,getStats(df7,6))))

        return AllDict
    
    except Exception as e:
        AllDict=dict.fromkeys(allKeyList1+allKeyList2,np.nan)
        return AllDict

In [125]:
# Kiindulási mappa, amiben a további almappák vannak
poemsPath="D:\Iskolai\Tananyagok\Szakdolgozat\Bsc\Versek"

rawIndex=0

df=pd.DataFrame()

# os.walk használata az összes almappa bejárásához
for rootFolder, Folder, Files in os.walk(poemsPath):
    for File in Files:
        if File.endswith('.xml'):  # Csak az XML kiterjesztésű fájlokat vesszük figyelembe
            completePath = os.path.join(rootFolder, File)

            # XML fájl beolvasása
            try:
                tree = ET.parse(completePath)
                root = tree.getroot()

                # Itt dolgozhatsz az XML tartalommal
                # Például: print(root.tag)

                functionList=[getAuthorName,getDiv,getLg,getL,getW,getPC,getRhymePair,getAlliteration]
                
                mainDict={}
                
                for function in functionList:
                    mainDict.update(function(completePath))    
                
                if rawIndex==0:                        

                    column_names=list(mainDict.keys())            

                    df = pd.DataFrame(columns=column_names)                   

                for item in mainDict:    

                    df.loc[rawIndex,item] = mainDict[item]

                rawIndex+=1

            except ET.ParseError:
                print(f'Hiba a(z) {File} XML fájl olvasásakor.')

#print(mainDictionary)

In [126]:
# df
reszlet=df.loc[0:59,'LmeanNW':'LvNlongS']
reszlet

,LmeanNW,LmaxNW,LminNW,LmedNW,Lq1NW,Lq3NW,LvNW,LmeanNS,LmaxNS,LminNS,...,Lq1NshortS,Lq3NshortS,LvNshortS,LmeanNlongS,LmaxNlongS,LminNlongS,LmedNlongS,Lq1NlongS,Lq3NlongS,LvNlongS
0,5.125,7,4,5.0,4.0,6.0,0.2,9.812,11,8,...,4.0,6.0,0.291,4.875,7,3,5.0,4.0,6.0,0.279
1,4.32,7,2,4.0,4.0,5.0,0.248,8.4,9,8,...,3.0,5.0,0.386,4.64,7,1,5.0,4.0,6.0,0.298
2,4.812,6,4,4.5,4.0,6.0,0.189,8.5,9,8,...,4.0,5.0,0.23,4.0,6,2,4.0,3.0,5.0,0.289
3,2.867,4,1,3.0,2.0,4.0,0.345,5.0,5,5,...,1.5,3.5,0.656,2.733,5,1,3.0,1.5,3.5,0.544
4,4.042,6,2,4.0,3.75,5.0,0.225,7.5,8,7,...,2.75,4.0,0.313,4.125,6,1,4.0,3.75,5.0,0.27
5,3.933,10,2,3.0,3.0,4.5,0.522,7.0,10,5,...,1.5,3.5,0.678,4.2,7,2,4.0,3.0,5.5,0.394
6,3.417,5,2,3.5,2.0,4.25,0.363,6.917,9,5,...,3.0,4.0,0.428,3.083,5,2,3.0,2.0,4.0,0.378
7,3.667,5,2,4.0,3.0,4.0,0.197,9.133,10,9,...,3.5,5.0,0.196,5.067,6,4,5.0,4.5,6.0,0.158
8,3.2,4,2,3.0,3.0,4.0,0.247,7.0,7,7,...,2.0,3.0,0.403,4.6,6,3,4.5,4.0,5.0,0.21
9,5.333,7,4,5.5,4.75,6.0,0.185,9.917,10,9,...,4.0,5.0,0.242,5.417,8,4,5.0,5.0,5.25,0.2
